In [3]:
import damask
import numpy as np 
import pandas as pd
import seaborn as sns

# 1. create input files (geometery, material, load)

In [6]:
# geometry file (3d periodic grid)

size = np.ones(3)*1e-5
cells = [16,16,16]
N_grains = 20
seeds = damask.seeds.from_random(size,N_grains,cells)
grid = damask.GeomGrid.from_Voronoi_tessellation(cells,size,seeds)
grid.save(f'Polycystal_{N_grains}_{cells[0]}x{cells[1]}x{cells[2]}')
grid

cells:  16 × 16 × 16
size:   1e-05 × 1e-05 × 1e-05 m³
origin: 0.0   0.0   0.0 m
# materials: 20

In [8]:
# material file 
mat = damask.ConfigMaterial.load('../DAMASK_yi/examples/config/phase/mechanical/plastic/phenopowerlaw_Al.yaml')

In [9]:
mat.

type: phenopowerlaw
references: ['W.F. Hosford et al., Acta Metallurgica 8(3):187-199, 1960, https://doi.org/10.1016/0001-6160(60)90127-9, fitted to Fig. 5 ([111] and [001])']
output: [xi_sl, gamma_sl]
N_sl: [12]
dot_gamma_0_sl: [7.5e-05]
n_sl: [20]
a_sl: [5.4]
xi_0_sl: [2690000.0]
xi_inf_sl: [67500000.0]
h_0_sl-sl: [281500000.0]
h_sl-sl: [1, 1, 5.123, 0.574, 1.123, 1.123, 1]

In [10]:
mat = damask.ConfigMaterial.load('../DAMASK_yi/examples/grid/material.yaml')
mat

homogenization:
  SX:
    N_constituents: 1
    mechanical: {type: pass}
phase:
  Aluminum:
    lattice: cF
    mechanical:
      output: [F, P, F_e, F_p, L_p, O]
      elastic: {type: Hooke, C_11: 106750000000.0, C_12: 60410000000.0, C_44: 28340000000.0}
      plastic:
        type: phenopowerlaw
        N_sl: [12]
        a_sl: [2.25]
        atol_xi: 1.0
        dot_gamma_0_sl: [0.001]
        h_0_sl-sl: [75000000.0]
        h_sl-sl: [1, 1, 1.4, 1.4, 1.4, 1.4, 1.4]
        n_sl: [20]
        output: [xi_sl]
        xi_0_sl: [31000000.0]
        xi_inf_sl: [63000000.0]
material:
- homogenization: SX
  constituents:
  - phase: Aluminum
    v: 1.0
    O: [1.0, 0.0, 0.0, 0.0]
- homogenization: SX
  constituents:
  - phase: Aluminum
    v: 1.0
    O: [0.7936696712125002, -0.28765777461664166, -0.3436487135089419, 0.4113964260949434]
- homogenization: SX
  constituents:
  - phase: Aluminum
    v: 1.0
    O: [0.3986143167493579, -0.7014883552495493, 0.2154871765709027, 0.5500781677772945]


In [ ]:
rnd = damask.Rotation.from_random(7)
fbr = damask.Rotation.from_fiber_component(crystal=[1,0],sample=[1,0],
                                           sigma=5.0,shape=5,degrees=True)
sph = damask.Rotation.from_spherical_component(center=damask.Rotation(),
                                               sigma=7.5,shape=3,degrees=True)
config = damask.ConfigMaterial()\
               .material_add(O=rnd,phase='phase_A',homogenization='SX')\
               .material_add(O=fbr,phase='phase_A',homogenization='SX')\
               .material_add(O=sph,phase='phase_B',homogenization='SX')
print(f'configuration is{" " if config.is_valid else " not "}valid\n')

config.save()
config

In [ ]:
# load file
def inversion(l,fill=0):
    return [inversion(i,fill) if isinstance(i,list) else\
            fill if i == 'x' else 'x' for i in l]
load_case = damask.Config(solver={'mechanical':'spectral_basic'},
                          loadstep=[])
F = [[1.05, 0 , 0 ],
     [   0,'x', 0 ],
     [   0, 0 ,'x']]

loadstep = {'boundary_conditions':{'mechanical':{'F':F,
                                                 'P':inversion(F)}},
                                   'discretization':{'t':10.,'N':40},'f_out':4}
load_case['loadstep'].append(loadstep)
dot_P = [[ 0 ,'x','x'],
         ['x','x','x'],
         ['x','x','x']]

loadstep = {'boundary_conditions':{'mechanical':{'dot_P':dot_P,
                                                 'dot_F':inversion(dot_P)}},
                                   'discretization':{'t':10.,'N':20}}
load_case['loadstep'].append(loadstep)
P = [[ 0 ,'x','x'],
     ['x', 0 ,'x'],
     ['x','x', 0 ]]

loadstep = {'boundary_conditions':{'mechanical':{'P':P,
                                                 'dot_F':inversion(P)}},
                                   'discretization':{'t':10.,'N':20}}
load_case['loadstep'].append(loadstep)
load_case.save('tension-hold-unload.yaml')
load_case

# 2. run simulation with DAMASK-grid solver

In [11]:
damask.util.run?

Signature:
damask.util.run(
    cmd: str,
    wd: str = './',
    env: Optional[Dict[str, str]] = None,
    timeout: Optional[int] = None,
) -> Tuple[str, str]
Docstring:
Run a command.

Parameters
----------
cmd : str
    Command to be executed.
wd : str, optional
    Working directory of process. Defaults to './'.
env : dict, optional
    Environment for execution.
timeout : integer, optional
    Timeout in seconds.

Returns
-------
stdout, stderr : (str, str)
    Output of the executed command.
File:      ~/workspace/DAMASK_yi/python/damask/util.py
Type:      function

# 3. process result and visualize

In [ ]:
result_file = 'add_field_data/20grains16x16x16_tensionX.hdf5'
result = damask.Result(result_file)
result.add_stress_Cauchy()
result.add_strain()

# list data (undeformed configuration)
result.add_equivalent_Mises('sigma')
result.add_equivalent_Mises('epsilon_V^0.0(F)')

# list data (undeformed configuration)
result.view(increments=0)

In [ ]:
result.export_simulation_setup()
grid_file = '20grains16x16x16.vti'
grid   = damask.Grid.load(grid_file)
# loop over all increments and store P(1,1) per grain and avg(F(1,1))
data = {g:pd.DataFrame() for g in grains}
for inc in result.get(['F','P']).values():
    P = inc['P']
    F = inc['F']
    for g in grains:
        points = grid.material.flatten(order='F')==g
        P_11 = P[points,0,0].flatten()
        F_11 = np.broadcast_to(np.average(F[:,0,0]),P_11.shape)
        x = pd.DataFrame({'F_11':F_11,'P_11':P_11})
        data[g] = pd.concat((data[g],x),ignore_index=True)

In [ ]:
for g in grains:
   plot = sns.lineplot(y='P_11',x='F_11',data=data[g])

fig = plot.get_figure()

In [ ]:
# export and visualize with paraview
result_select = result.view_in_times()
result_select.export()